# Walker2D

You aim in this task is to train the agent to win in Walker2D game with Actor-Critic, Advantage Actor Critic (A2C), Trust-region Policy Optimization (TRPO) or Proximal Policy Optimization (PPO).
To solve the task feel free to transform the state and reward from the environment.

**Scoring**: Calculating the average reward for 50 episodes. You goal is to gain more than 1000 points.

**Submission format**: send you notebook and trained model in **zipped** folder.

In [ ]:
!pip install PyBullet >> None

In [ ]:
!pip install gym

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from collections import UserDict

import gym
import gym.envs.registration

# Do this before importing pybullet_envs (adds an extra property env_specs as a property to the registry, so it looks like the <0.26 envspec version)
registry = UserDict(gym.envs.registration.registry)
registry.env_specs = gym.envs.registration.registry
gym.envs.registration.registry = registry

import pybullet_envs
# racecar_env = gym.make('RacecarBulletEnv-v0')

In [ ]:
# import pybullet_envs
# from gym import make
import numpy as np
import torch
from torch import nn
from torch.distributions import Normal
from torch.nn import functional as F
from torch.optim import Adam
import random
import os

> Probably it's a good idea to check out doc [Walker2D](https://mgoulao.github.io/gym-docs/environments/mujoco/walker2d/)

# Train

In [ ]:
ENV_NAME = "Walker2DBulletEnv-v0"

LAMBDA = 0.95
GAMMA = 0.99

ACTOR_LR = 2e-4
CRITIC_LR = 1e-4

CLIP = 0.2
ENTROPY_COEF = 1e-2
BATCHES_PER_UPDATE = 64
BATCH_SIZE = 64

MIN_TRANSITIONS_PER_UPDATE = 2048
MIN_EPISODES_PER_UPDATE = 4

ITERATIONS = 1000

In [ ]:
def compute_lambda_returns_and_gae(trajectory):
    lambda_returns = []
    gae = []
    last_lr = 0.
    last_v = 0.
    for _, _, r, _, v in reversed(trajectory):
        ret = r + GAMMA * (last_v * (1 - LAMBDA) + last_lr * LAMBDA)
        last_lr = ret
        last_v = v
        lambda_returns.append(last_lr)
        gae.append(last_lr - v)

    # Each transition contains state, action, old action probability, value estimation and advantage estimation
    return [(s, a, p, v, adv) for (s, a, _, p, _), v, adv in zip(trajectory, reversed(lambda_returns), reversed(gae))]

Reference:
> [UNIT 6. ACTOR CRITIC METHODS WITH ROBOTICS ENVIRONMENTS](https://huggingface.co/deep-rl-course/unit6/introduction?fw=pt)

In [ ]:
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size=256):
        super().__init__()
        # Advice: use same log_sigma for all states to improve stability
        # You can do this by defining log_sigma as nn.Parameter(torch.zeros(...))
        ''' YOUR CODE HERE '''
        self.model = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.ReLU(),

            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),

            nn.Linear(hidden_size, action_dim)
        )
        self.sigma = nn.Parameter(
                torch.zeros(action_dim)
        )

    def compute_proba(self, state, action):
        # Returns probability of action according to current policy and distribution of actions
        ''' YOUR CODE HERE '''
        mu = self.model(state)
        distribution = Normal(mu, torch.exp(self.sigma))
        # action = distribution.sample()
        probability = torch.exp(
            distribution.log_prob(action).sum(-1)
        )
        # probability = distribution.log_prob(action)

        return probability, distribution


    def act(self, state):
        # Returns an action (with tanh), not-transformed action (without tanh) and distribution of non-transformed actions
        # Remember: agent is not deterministic, sample actions from distribution (e.g. Gaussian)
        ''' YOUR CODE HERE '''
        mu = self.model(state)
        distribution = Normal(mu, torch.exp(self.sigma))
        action = distribution.sample()
        actionWithTanh = torch.tanh(action)

        return actionWithTanh, action, distribution

In [ ]:
class Critic(nn.Module):
    def __init__(self, state_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ELU(),
            nn.Linear(256, 256),
            nn.ELU(),
            nn.Linear(256, 1)
        )

    def get_value(self, state):
        return self.model(state)

In [ ]:
class PPO:
    def __init__(self, state_dim, action_dim):
        self.actor = Actor(state_dim, action_dim)
        self.critic = Critic(state_dim)
        self.actor_optim = Adam(self.actor.parameters(), ACTOR_LR)
        self.critic_optim = Adam(self.critic.parameters(), CRITIC_LR)

    def update(self, trajectories):
        transitions = [t for traj in trajectories for t in traj]     # Turn a list of trajectories into list of transitions
        state, action, old_prob, target_value, advantage = zip(*transitions)
        state = np.array(state)
        action = np.array(action)
        old_prob = np.array(old_prob)
        target_value = np.array(target_value)
        advantage = np.array(advantage)
        advnatage = (advantage - advantage.mean()) / (advantage.std() + 1e-8)


        for _ in range(BATCHES_PER_UPDATE):
            idx = np.random.randint(0, len(transitions), BATCH_SIZE) # Choose random batch
            s = torch.tensor(state[idx]).float()
            a = torch.tensor(action[idx]).float()
            op = torch.tensor(old_prob[idx]).float()                 # Probability of the action in state s.t. old policy
            v = torch.tensor(target_value[idx]).float()              # Estimated by lambda-returns
            adv = torch.tensor(advantage[idx]).float()               # Estimated by generalized advantage estimation

            ''' YOUR CODE HERE '''
            # TODO: Update actor here
            # 1) Ratios
            currProb, distribution = self.actor.compute_proba(s, a)
            ratio = currProb / op
            # 2) Actors Loss
            surrogateLoss = ratio * adv
            entropy = distribution.entropy().mean()
            clippedSurrogateLoss = (torch.clamp(ratio, 1.0 - CLIP, 1.0 + CLIP) * adv)

            actor_loss = -torch.mean(torch.minimum(surrogateLoss, clippedSurrogateLoss))
            # 3) Train Actor
            self.actor_optim.zero_grad()
            actor_loss.backward()
            self.actor_optim.step()

            # TODO: Update critic here
            # 1) Get Critic's value
            value = self.critic.get_value(s).flatten()
            # 2) Actors Loss
            critic_loss = F.smooth_l1_loss(value, v)

            # 3) Train Critic
            self.critic_optim.zero_grad()
            critic_loss.backward(retain_graph=True)
            self.critic_optim.step()


    def get_value(self, state):
        with torch.no_grad():
            state = torch.tensor(np.array([state])).float()
            value = self.critic.get_value(state)
        return value.cpu().item()

    def act(self, state):
        with torch.no_grad():
            state = torch.tensor(np.array([state])).float()
            action, pure_action, distr = self.actor.act(state)
            prob = torch.exp(distr.log_prob(pure_action).sum(-1))
        return action.cpu().numpy()[0], pure_action.cpu().numpy()[0], prob.cpu().item()

    def save(self):
        torch.save(self.actor, "agent.pkl")

In [ ]:
def evaluate_policy(env, agent, episodes=5):
    returns = []
    for _ in range(episodes):
        done = False
        state = env.reset()
        total_reward = 0.

        while not done:
            state, reward, done, _ = env.step(agent.act(state)[0])
            total_reward += reward
        returns.append(total_reward)
    return returns


def sample_episode(env, agent):
    s = env.reset()
    d = False
    trajectory = []
    while not d:
        a, pa, p = agent.act(s)
        v = agent.get_value(s)
        ns, r, d, _ = env.step(a)
        trajectory.append((s, pa, r, p, v))
        s = ns
    return compute_lambda_returns_and_gae(trajectory)

In [ ]:
#env = make(ENV_NAME)
env = gym.make(ENV_NAME)
ppo = PPO(state_dim=env.observation_space.shape[0], action_dim=env.action_space.shape[0])
state = env.reset()
episodes_sampled = 0
steps_sampled = 0

for i in range(ITERATIONS):
    trajectories = []
    steps_ctn = 0

    while len(trajectories) < MIN_EPISODES_PER_UPDATE or steps_ctn < MIN_TRANSITIONS_PER_UPDATE:
        traj = sample_episode(env, ppo)
        steps_ctn += len(traj)
        trajectories.append(traj)
    episodes_sampled += len(trajectories)
    steps_sampled += steps_ctn

    ppo.update(trajectories)

    if (i + 1) % (ITERATIONS//100) == 0:
        rewards = evaluate_policy(env, ppo, 5)
        print(f"Step: {i+1}, Reward mean: {np.mean(rewards)}, Reward std: {np.std(rewards)}, Episodes: {episodes_sampled}, Steps: {steps_sampled}")
        ppo.save()

/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that

Step: 10, Reward mean: 13.490421574568026, Reward std: 2.1758189853793044, Episodes: 1944, Steps: 20521
Step: 20, Reward mean: 12.40257092276559, Reward std: 1.4240354023820183, Episodes: 4067, Steps: 41033
Step: 30, Reward mean: 14.335262117974633, Reward std: 2.882915793335888, Episodes: 5882, Steps: 61572
Step: 40, Reward mean: 12.868770925158461, Reward std: 1.834846839880667, Episodes: 7797, Steps: 82085
Step: 50, Reward mean: 12.609328412076866, Reward std: 1.4526973660943707, Episodes: 9698, Steps: 102618
Step: 60, Reward mean: 13.107310694982587, Reward std: 1.993354418838299, Episodes: 11647, Steps: 123150
Step: 70, Reward mean: 8.943115792915052, Reward std: 1.4436820017767429, Episodes: 13496, Steps: 143669
Step: 80, Reward mean: 4.056390968292544, Reward std: 0.9729407159018734, Episodes: 15476, Steps: 164198
Step: 90, Reward mean: 5.860691002358509, Reward std: 1.070550033261193, Episodes: 17469, Steps: 184722
Step: 100, Reward mean: 5.00091380233207, Reward std: 1.3950112

# Agent

In [ ]:
class Agent:
    def __init__(self):
        self.model = torch.load(__file__[:-8] + "/agent.pkl")

    def act(self, state):
        with torch.no_grad():
            state = torch.tensor(np.array(state)).float()
            ''' YOUR CODE HERE '''
            # state = torch.tensor(state, device=device, dtype=torch.float32)
            action = self.model(state)
            action = np.argmax(action.numpy())
        return action #None

    def reset(self):
        pass